In [ ]:
import json
import os
import tensorflow as tf
import tarfile
import urllib.request

In [ ]:
tf.enable_eager_execution()

In [ ]:
_root = '.'
_data_root = _root + '/data'

In [ ]:
archive_file = os.path.join(_data_root, 'train_val2019.tar.gz')

if os.path.isfile(archive_file):
    print(archive_file + ' file already exists, skipping...')
else:
    urllib.request.urlretrieve('http://www.vision.caltech.edu/~gvanhorn/datasets/inaturalist/fgvc6_competition/train_val2019.tar.gz', archive_file)
    tar = tarfile.open(archive_file, "r:gz")
    tar.extractall(_data_root)
    tar.close()

In [ ]:
IMG_SIZE = 224
SUPERCATEGORY = 'Plants'

path = os.path.join(_data_root, 'train_val2019', SUPERCATEGORY)
image_labels = {}
i = 0
for name in os.listdir(path):
    image_labels[name] = i
    i += 1

with open(os.path.join(_data_root, 'labels.json'), "w") as f:
    f.write(json.dumps(image_labels))

for annotation in ('train', 'eval'):
    output_file = os.path.join(_data_root, annotation + '.tfrecord')
    if os.path.isfile(output_file):
        print(output_file + ' file already exists, skipping...')
    else:
        with tf.io.TFRecordWriter(output_file) as writer:
            with open(os.path.join(_data_root, annotation + '2019.json')) as json_file:
                data = json.load(json_file)
                x = 0
                for img in data['images']:
                    x+=1
                    if x>50:
                        break
                    file_name = img['file_name']
                    splitted_file_name = file_name.split('/')
                    if splitted_file_name[1] == SUPERCATEGORY:
                        labels = str(image_labels[splitted_file_name[2]])
                        image_string = open(os.path.join(_data_root, file_name), 'rb').read()
                        image = tf.image.decode_image(image_string, dtype=tf.dtypes.uint8)
                        image_resized = tf.cast(tf.image.resize_images(image, size=[IMG_SIZE,IMG_SIZE],
                                                                       method=tf.image.ResizeMethod.AREA),
                                                tf.uint8)
                        image_string = tf.image.encode_jpeg(image_resized)
                        feature = {
                            'labels': tf.train.Feature(bytes_list=tf.train.BytesList(
                                value=[tf.compat.as_bytes(labels)])),
                            'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(
                                value=[image_string.numpy()])),
                        }

                        tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
                        writer.write(tf_example.SerializeToString())
                        